# Proyecto Data Science "Salarios de los cientificos de datos"

## Abstract

Desde que descubrí el mundo de la ciencia de datos, siempre he tenido una curiosidad insaciable por entender cómo las variables influyen en los resultados. En particular, me he preguntado: ¿Cuáles son las variables que más se correlacionan con el nivel de ingresos de un científico de datos? ¿El tamaño de la empresa tiene algo que ver con el nivel de ingresos?

Para responder a estas preguntas, decidí embarcarme en un proyecto de wrangling de datos. Comencé con un dataframe de salarios de científicos de datos, pero sabía que necesitaba más información para enriquecer mis datos. Así que decidí utilizar una API para obtener datos adicionales.

La API me proporcionó información valiosa sobre las empresas, como su tamaño y su ubicación. Con estos nuevos datos, pude enriquecer mi dataframe original y obtener una visión más completa del panorama.

Una vez que tuve todos los datos que necesitaba, comencé el proceso de wrangling. Limpié los datos, manejé los valores perdidos y outliers, y preparé los datos para el análisis.

Finalmente, llegó el momento de responder a mis hipótesis. Utilicé técnicas estadísticas para identificar las variables que más se correlacionaban con el nivel de ingresos. También analicé si el tamaño de la empresa tenía algún efecto en el nivel de ingresos.

Este proyecto fue un viaje increíblemente enriquecedor. No sólo me permitió responder a mis preguntas, sino que también me dio una visión más profunda del campo de la ciencia de datos. Me hizo darme cuenta de que quiero seguir creciendo y aprendiendo en este campo, y estoy emocionado por lo que vendrá a continuación.

***

Instalo el paquete python-dotenv en el entorno Python actual. Este paquete es útil para manejar archivos .env, que son una forma común y segura de almacenar configuraciones secretas o específicas del entorno (como claves API o contraseñas) para tu aplicación.

In [1]:
!pip install python-dotenv

Estoy utilizando este código para cargar dos claves API desde un archivo .env y almacenarlas en las variables api_key1 y api_key2. Esto es útil porque me permite mantener estas claves API fuera del código fuente, lo cual es una buena práctica para proteger información sensible. Ah si mismo el archivo en el que esta almacenadas las APIKEY esta incluido en un .gitignore.

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key1 = os.getenv("API_KEY_DI")
api_key2 = os.getenv("API_KEY_BEN")

In [3]:
# Importación de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import requests

# Carga del dataset
dataSet = '../data/jobs_data.csv'
df = pd.read_csv(dataSet)

# Resumen general del dataset
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   title         220 non-null    object 
 1   company_name  220 non-null    object 
 2   location      220 non-null    object 
 3   description   220 non-null    object 
 4   salaries      0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 8.7+ KB
None


Este Dataframe fue obtenido en el trabajo anterior de conexiones con APIs.

In [4]:
df

,title,company_name,location,description,salaries
0,Data Scientist,"Audible, Inc.","Newark, NJ",Good storytelling starts with great listening....,NaN
1,Data Scientist,Internal Revenue Service,"Huntsville, AL (+36 others)","Click on ""Learn more about this agency"" button...",NaN
2,2024 University Graduate - Data Science Engine...,Adobe,"Biloxi, MS",Our Company\r\n\r\nChanging the world through ...,NaN
3,"Senior Data Scientist, Machine Learning (NLP)",Discord,Anywhere,Discord is looking for experienced and passion...,NaN
4,"Director, Data Science - Retail Media+",The Home Depot,"Atlanta, GA",Position Purpose:\r\n\r\nWe are actively seeki...,NaN
...,...,...,...,...,...
215,"Senior Data Scientist, Product Growth",Jerry,"Detroit, MI",We'd love to hear from you if you like:\r\n• M...,NaN
216,(Intern) Enterprise Analytics Office Data Scie...,Nationwide Insurance and Financial Services,"Hammond, IN",As a team member in the Finance and Internal A...,NaN
217,Staff Data Scientist,MongoDB,"San Francisco, CA",The worldwide data management software market ...,NaN
218,Postdoctoral Scholar,"The University of Chicago, Data Science Institute","Chicago, IL",The Data Science Institute (DSI) at the Univer...,NaN


In [7]:
# Define la URL
url = 'https://serpapi.com/search'

# Lista de todos los estados en los Estados Unidos
estados = ['Nueva York', 'Miami', 'Los Angeles', 'Las Vegas', 'Hawaii', 'Washington DC', 'Boston', 'Orlando', 'San Francisco', 'Chicago', 'Atlanta', 'Filadelfia', 'Seattle', 'Dallas', 'Austin', 'Houston', 'Denver', 'Detroit', 'Minneapolis', 'Phoenix', 'San Diego', 'St. Louis', 'Tampa', 'Baltimore', 'Pittsburgh', 'Portland', 'Charlotte', 'Indianapolis', 'Kansas City', 'New Orleans', 'Cleveland', 'Sacramento', 'Cincinnati', 'Milwaukee', 'Honolulu', 'Salt Lake City', 'Raleigh', 'Nashville', 'Buffalo', 'Richmond','Louisville','Providence','Oklahoma City','Jacksonville','Memphis','Columbus','Las Vegas','Albuquerque','Tucson','Fresno']


# Inicializa una lista vacía para almacenar los datos de los trabajos
jobs_data = []

# Haz una solicitud GET a la API para cada estado
for estado in estados:
    params = {
        'engine': 'google_jobs',
        'q': 'data science',  # Reemplaza esto con el título del trabajo que estás buscando
        'location': estado,  # Cambia la ubicación a cada estado
        'api_key': api_key2 ,  # Esta es la clave de API de SerpApi
        'num': 10  # Solicita 10 resultados por página
    }

    # Haz una solicitud GET a la API
    response = requests.get(url, params=params)

    # Verifica si 'jobs_results' está en la respuesta
    if 'jobs_results' in response.json():
        # Extrae la información relevante de los resultados de la búsqueda
        for job in response.json()['jobs_results']:
            job_info = {
                'title': job.get('title'),
                'company_name': job.get('company_name'),
                'location': job.get('location'),
                'description': job.get('description'),
                # Asegúrate de que el campo 'salaries' exista antes de intentar acceder a él
                'salaries': job.get('salaries') if 'salaries' in job else None,
            }
            jobs_data.append(job_info)
    else:
        print(f"No se encontraron resultados de trabajos para {estado}")

# Crea un DataFrame con los datos
df = pd.DataFrame(jobs_data)

# Guarda el DataFrame como un archivo CSV
df.to_csv('..data/jobs_data2.csv', index=False)    

No se encontraron resultados de trabajos para Nueva York
No se encontraron resultados de trabajos para Filadelfia


OSError: Cannot save file into a non-existent directory: '..data'

In [6]:
import numpy as np

# Definimos una función para extraer el número después del símbolo '$'
def extraer_numero(texto):
    match = re.search("\$([0-9,\.]+)", texto)
    if match is not None:
        return match.group(1)
    else:
        return np.nan

# Aplicamos la función a la columna 'description' y creamos una nueva columna 'salary'
df['salary'] = df['description'].apply(extraer_numero)

df

KeyError: 'description'

In [ ]:
import re

for i in range(len(df)):
    match = re.search("\$", df.description[i])
    if match is not None:
        print(list(match.span())[0])
    else:
        print("No se encontró el símbolo '$'")

No se encontró el símbolo '$'
No se encontró el símbolo '$'
2820
2150
No se encontró el símbolo '$'
4970
No se encontró el símbolo '$'
2457
3552
3878
81
No se encontró el símbolo '$'
No se encontró el símbolo '$'
2820
No se encontró el símbolo '$'
4970
No se encontró el símbolo '$'
3878
2237
No se encontró el símbolo '$'
3552
735
No se encontró el símbolo '$'
81
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró el símbolo '$'
4920
4970
No se encontró el símbolo '$'
2150
No se encontró el símbolo '$'
81
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró el símbolo '$'
5705
No se encontró el símbolo '$'
81
No se encontró el símbolo '$'
1919
No se encontró el símbolo '$'
No se encontró el símbolo '$'
3890
4065
5908
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró el símbolo '$'
4100
No se encontró el símbolo '$'
No se encontró el símbolo '$'
No se encontró

In [ ]:
# inspeccion de donde puede estar el saladio dentro de descripcion
num_fila = 215  # El número de la fila que quieres seleccionar
nombre_columna = 'description'  # El nombre de la columna que quieres seleccionar

# Usamos at para seleccionar el valor
valor = df.at[num_fila, nombre_columna]

# Ahora valor es una cadena de texto. Podemos extraer una subcadena así:
inicio = 0  # La posición inicial desde donde quieres extraer
fin = 3000  # La posición final hasta donde quieres extraer

# Usamos la indexación de cadenas de texto para extraer la subcadena
subcadena = valor[inicio:fin]

print(subcadena)

We'd love to hear from you if you like:
• Making a big impact with a Forbes Top Startup Employer
• Working on products that have traction (40X revenue growth in 4 years | #1 rated app in the insurance comparison category...
• Solving problems in a huge market ($2T market size)
• Working closely with serial entrepreneurs and seasoned leaders who have scaled companies like Robinhood, Amazon, LinkedIn, Wayfair, SoFi, Microsoft, etc.

About the opportunity:

We are looking for a Senior Data Scientist to join our central data team and partner with one of our emerging product groups. Helping everyday, hard working Americans save time and money on their cars and creating a world class experience is what drives every decision we make as a company. Since launching our mobile app in 2019, we have amassed over 4M customers, expanded our product offerings to multiple categories and scaled our team 10X. Our data team fuels all of our business and product decisions through delivering analytical insi

In [ ]:
df.isnull()

NameError: name 'df' is not defined